<a href="https://colab.research.google.com/github/garedo/Machine-Learning/blob/main/Estudo_de_Caso_Elei%C3%A7%C3%B5es_no_Brasil_Minicurso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de Caso: Eleições no Brasil

O Repositório de dados eleitorais é uma compilação de informações reais das eleições de 2014 e 2018, voltada para pesquisadores, imprensa e pessoas interessadas em analisar os dados de eleitorado, candidaturas, resultados e prestação de contas.

Neste Estudo de Caso, iremos analisar as informações acerca do perfil dos candidatos nas eleições de 2014 e 2018 no âmbito estadual e federal, declarações de bens e dados sobre os partidos, as coligações e as vagas por cargo e por unidade eleitoral.

Os dados foram retirados do [Repositório de Dados Eleitorais do TSE](http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1).

Abaixo as bases de dados das eleições de 2014 e 2018 são baixadas do repositório de dados do TSE:

In [ ]:
sh = """
if [ ! -d eleicoes ]; then
  mkdir eleicoes
fi

# Baixa o arquivo que contém as informações dos candidatos de 2014
if [ ! -f eleicoes/consulta_candidato_2014_BRASIL.csv ]; then 
  wget https://github.com/savioteles/big_data/raw/master/etl/datasets/consulta_cand_2014_BRASIL.csv
  mv consulta_cand_2014_BRASIL.csv eleicoes
fi

# Baixa o arquivo que contém as informações dos bens dos candidatos de 2014
if [ ! -f eleicoes/bem_candidato_2014_BRASIL.csv ]; then 
  wget https://github.com/savioteles/big_data/raw/master/etl/datasets/bem_candidato_2014_BRASIL.csv
  mv bem_candidato_2014_BRASIL.csv eleicoes
fi

# Baixa o arquivo que contém as informações dos candidatos de 2018
if [ ! -f eleicoes/consulta_cand_2018_BRASIL.csv ]; then 
  wget https://github.com/savioteles/big_data/raw/master/etl/datasets/consulta_cand_2018_BRASIL.csv
  mv consulta_cand_2018_BRASIL.csv eleicoes
fi

# Baixa o arquivo que contém as informações dos bens dos candidatos de 2018
if [ ! -f eleicoes/bem_candidato_2018_BRASIL.csv ]; then 
  wget https://github.com/savioteles/big_data/raw/master/etl/datasets/bem_candidato_2018_BRASIL.csv
  mv bem_candidato_2018_BRASIL.csv eleicoes
fi
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

--2022-10-25 22:57:14--  https://github.com/savioteles/big_data/raw/master/etl/datasets/consulta_cand_2014_BRASIL.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/savioteles/big_data/master/etl/datasets/consulta_cand_2014_BRASIL.csv [following]
--2022-10-25 22:57:15--  https://raw.githubusercontent.com/savioteles/big_data/master/etl/datasets/consulta_cand_2014_BRASIL.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17108987 (16M) [text/plain]
Saving to: ‘consulta_cand_2014_BRASIL.csv’

consulta_cand_2014_ 100%[===================>]  16.32M  --.-KB/s    in 0.07s   

2022-10-25 22:57:15 (221

## Iniciando o PySpark


In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('Eleicoes').master("local[*]").getOrCreate()

## Primeira etapa: carregar os arquivos

Nessa segunda etapa você deve carregar os quatro arquivos abaixos, utilizando o **Spark**:

- file:///content/eleicoes/bem_candidato_2014_BRASIL.csv: você deve montar o esquema na leitura do arquivo
- file:///content/eleicoes/bem_candidato_2018_BRASIL.csv: você deve montar o esquema na leitura do arquivo
- file:///content/eleicoes/consulta_cand_2014_BRASIL.csv: o esquema de dados deve ser inferido automaticamente (inferSchema=True)
- file:///content/eleicoes/consulta_cand_2018_BRASIL.csv: o esquema de dados deve ser inferido automaticamente (inferSchema=True)

**Dicas:**

- Arquivos estão com codificação Latin1 (encoding='Latin1')
- Separador dos arquivos é ; (ponto e vírgula)
- Os arquivos possuem cabeçalho
- Os esquemas de dados dos arquivos bem_candidato_2014_BRASIL.csv e bem_candidato_2018_BRASIL.csv são iguais, ou seja, é preciso montar apenas um esquema com ScructType e aproveitar na leitura dos dois arquivo.

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
bem_candidato_schema = StructType([StructField("DT_GERACAO", DateType(), True),
                    StructField("HH_GERACAO", StringType(), True),
                    StructField("ANO_ELEICAO", IntegerType(), True),
                    StructField("CD_TIPO_ELEICAO", IntegerType(), True),
                    StructField("NM_TIPO_ELEICAO", StringType(), True),
                    StructField("CD_ELEICAO", IntegerType(), True),
                    StructField("DS_ELEICAO", StringType(), True),
                    StructField("DT_ELEICAO", DateType(), True),
                    StructField("SG_UF", StringType(), True),
                    StructField("SG_UE", StringType(), True),
                    StructField("NM_UE", StringType(), True),
                    StructField("SQ_CANDIDATO", LongType(), True),
                    StructField("NR_ORDEM_CANDIDATO", IntegerType(), True),
                    StructField("CD_TIPO_BEM_CANDIDATO", IntegerType(), True),
                    StructField("DS_TIPO_BEM_CANDIDATO", StringType(), True),
                    StructField("DS_BEM_CANDIDATO", StringType(), True),
                    StructField("VR_BEM_CANDIDATO", StringType(), True),
                    StructField("DT_ULTIMA_ATUALIZACAO", DateType(), True),
                    StructField("HH_ULTIMA_ATUALIZACAO", StringType(), True)])

Abaixo faça a leitura dos arquivos com informações dos bens dos candidatos em 2014 e 2018

In [ ]:
# Leitura dos arquivos com informações dos bens dos candidatos em 2014 e 2018
bem_candidato_2014_df = spark.read.csv('file:///content/eleicoes/bem_candidato_2014_BRASIL.csv', \
                                       header=True, encoding='Latin1', \
                                       sep=';', schema=bem_candidato_schema, \
                                       dateFormat='dd/MM/yyyy')
bem_candidato_2018_df = spark.read.csv('file:///content/eleicoes/bem_candidato_2018_BRASIL.csv', header=True, encoding='Latin1', sep=';', schema=bem_candidato_schema, dateFormat='dd/MM/yyyy')

Faça a leitura dos arquivos da candidatura dos anos de 2014 e 2018 com inferSchema do Spark

In [ ]:
# Leitura dos arquivos da candidatura dos anos de 2014 e 2018
consulta_cand_2014_df = spark.read.csv('file:///content/eleicoes/consulta_cand_2014_BRASIL.csv', \
                                       header=True, encoding='Latin1', \
                                       sep=';', inferSchema=True, \
                                       timestampFormat='dd/MM/yyyy')
consulta_cand_2018_df = spark.read.csv('file:///content/eleicoes/consulta_cand_2018_BRASIL.csv', header=True, encoding='Latin1', sep=';', inferSchema=True, timestampFormat='dd/MM/yyyy')

## Segunda etapa: fazer o tratamento nos dados

Nesta etapa devemos substituir os valores igual a #NULO# por ''(vazio) nas colunas 'NM_SOCIAL_CANDIDATO' e 'DS_SIT_TOT_TURNO' dos DataFrames consulta_cand_2014_df e consulta_cand_2018_df.

In [ ]:
# candidatos 2014
consulta_cand_2014_df = consulta_cand_2014_df.\
withColumn('DS_SIT_TOT_TURNO', regexp_replace('DS_SIT_TOT_TURNO', '#NULO#', ''))

consulta_cand_2014_df = consulta_cand_2014_df.\
withColumn('NM_SOCIAL_CANDIDATO', regexp_replace('NM_SOCIAL_CANDIDATO', '#NULO#', ''))

# candidatos 2018
consulta_cand_2018_df = consulta_cand_2018_df.withColumn('DS_SIT_TOT_TURNO', regexp_replace('DS_SIT_TOT_TURNO', '#NULO#', ''))
consulta_cand_2018_df = consulta_cand_2018_df.withColumn('NM_SOCIAL_CANDIDATO', regexp_replace('NM_SOCIAL_CANDIDATO', '#NULO#', ''))

Além disso, temos que substituir os valores igual a #NULO# por ''(vazio) na coluna 'DS_BEM_CANDIDATO' dos DataFrames bem_candidato_2014_df e bem_candidato_2018_df.

In [ ]:
# bens dos candidatos de 2014
bem_candidato_2014_df = bem_candidato_2014_df.withColumn('DS_BEM_CANDIDATO', regexp_replace('DS_BEM_CANDIDATO', '#NULO#', ''))
# bens dos candidatos de 2018
bem_candidato_2018_df = bem_candidato_2018_df.withColumn('DS_BEM_CANDIDATO', regexp_replace('DS_BEM_CANDIDATO', '#NULO#', ''))

## Terceira Etapa: realizar consultas nos dados

Nesta etapa iremos fazer diversas consultas nos dados do TSE utilizando a API do Spark e SparkSQL.

### **Quais candidatos foram eleitos em 2014?**

Apresente na tela os campos 'NM_CANDIDATO', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'DS_CARGO' dos candidatos eleitos. Candidatos eleitos são aqueles com CD_SIT_TOT_TURNO igual a 1, 2 ou 3.

In [ ]:
# Quais candidatos foram eleitos em 2014?
eleitos_2014 = consulta_cand_2014_df\
.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3'))\
.select('NM_CANDIDATO', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'DS_CARGO')

eleitos_2014.show(5, truncate=False)

+----------------------------+----------------+------------+-----------------+
|NM_CANDIDATO                |NR_CPF_CANDIDATO|SQ_CANDIDATO|DS_CARGO         |
+----------------------------+----------------+------------+-----------------+
|ANTÔNIO WALDEZ GÓES DA SILVA|12617555291     |30000000088 |GOVERNADOR       |
|LUCÍLVIO GIRÃO SALES        |10493441387     |60000000689 |DEPUTADO ESTADUAL|
|SILAS CAMARA                |13512951287     |40000000360 |DEPUTADO FEDERAL |
|CARLOS MATOS LIMA           |23265388387     |60000000554 |DEPUTADO ESTADUAL|
|GUSTAVO REIS FERREIRA       |07615622735     |190000001611|DEPUTADO ESTADUAL|
+----------------------------+----------------+------------+-----------------+
only showing top 5 rows



In [ ]:
consulta_cand_2014_df.createOrReplaceTempView("consulta_cand_2014")
consulta_cand_2018_df.createOrReplaceTempView("consulta_cand_2018")
bem_candidato_2014_df.createOrReplaceTempView("bem_candidato_2014")
bem_candidato_2018_df.createOrReplaceTempView("bem_candidato_2018")

In [ ]:
spark.sql("select NM_CANDIDATO, NR_CPF_CANDIDATO, SQ_CANDIDATO, DS_CARGO \
from consulta_cand_2014 where CD_SIT_TOT_TURNO in ('1','2','3')").show(5)

+--------------------+----------------+------------+-----------------+
|        NM_CANDIDATO|NR_CPF_CANDIDATO|SQ_CANDIDATO|         DS_CARGO|
+--------------------+----------------+------------+-----------------+
|ANTÔNIO WALDEZ GÓ...|     12617555291| 30000000088|       GOVERNADOR|
|LUCÍLVIO GIRÃO SALES|     10493441387| 60000000689|DEPUTADO ESTADUAL|
|        SILAS CAMARA|     13512951287| 40000000360| DEPUTADO FEDERAL|
|   CARLOS MATOS LIMA|     23265388387| 60000000554|DEPUTADO ESTADUAL|
|GUSTAVO REIS FERR...|     07615622735|190000001611|DEPUTADO ESTADUAL|
+--------------------+----------------+------------+-----------------+
only showing top 5 rows



### **Quais candidatos foram eleitos em 2018?**

Apresente na tela os campos 'NM_CANDIDATO', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'DS_CARGO' dos candidatos eleitos. Candidatos eleitos são aqueles com CD_SIT_TOT_TURNO igual a 1, 2 ou 3.

In [ ]:
# Quais candidatos foram eleitos em 2018?
eleitos_2018 = consulta_cand_2018_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3')).select('NM_CANDIDATO', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'DS_CARGO', 'ST_REELEICAO')
eleitos_2018.show(5, truncate=False)

+-------------------------------+----------------+------------+-----------------+------------+
|NM_CANDIDATO                   |NR_CPF_CANDIDATO|SQ_CANDIDATO|DS_CARGO         |ST_REELEICAO|
+-------------------------------+----------------+------------+-----------------+------------+
|GILSON MARQUES VIEIRA          |824207939       |240000600256|DEPUTADO FEDERAL |N           |
|ANTONIO ARNALDO ALVES DE MELO  |5534640278      |100000609132|DEPUTADO ESTADUAL|N           |
|DANIEL TRZECIAK DUARTE         |1297812077      |210000604795|DEPUTADO FEDERAL |N           |
|CASSIO ANTÔNIO FERREIRA SOARES |4245226676      |130000613113|DEPUTADO ESTADUAL|S           |
|MARIA MARGARIDA MARTINS SALOMÃO|13521039668     |130000626964|DEPUTADO FEDERAL |S           |
+-------------------------------+----------------+------------+-----------------+------------+
only showing top 5 rows



In [ ]:
spark.sql("select NM_CANDIDATO, NR_CPF_CANDIDATO, SQ_CANDIDATO, DS_CARGO, ST_REELEICAO \
from consulta_cand_2018 where CD_SIT_TOT_TURNO in ('1','2','3')")\
.show(5, truncate=False)

+-------------------------------+----------------+------------+-----------------+------------+
|NM_CANDIDATO                   |NR_CPF_CANDIDATO|SQ_CANDIDATO|DS_CARGO         |ST_REELEICAO|
+-------------------------------+----------------+------------+-----------------+------------+
|GILSON MARQUES VIEIRA          |824207939       |240000600256|DEPUTADO FEDERAL |N           |
|ANTONIO ARNALDO ALVES DE MELO  |5534640278      |100000609132|DEPUTADO ESTADUAL|N           |
|DANIEL TRZECIAK DUARTE         |1297812077      |210000604795|DEPUTADO FEDERAL |N           |
|CASSIO ANTÔNIO FERREIRA SOARES |4245226676      |130000613113|DEPUTADO ESTADUAL|S           |
|MARIA MARGARIDA MARTINS SALOMÃO|13521039668     |130000626964|DEPUTADO FEDERAL |S           |
+-------------------------------+----------------+------------+-----------------+------------+
only showing top 5 rows



### **Qual o número de eleitos por partido em 2014 ordenando a saída pelo número de eleitos?**

Candidatos eleitos são aqueles com CD_SIT_TOT_TURNO igual a 1, 2 ou 3. O partido é definido pela coluna NM_PARTIDO.

In [ ]:
# Qual o número de eleitos por partido em 2014 ordenando a saída pelo número de eleitos?
consulta_cand_2014_df\
.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3'))\
.groupBy(['NM_PARTIDO']).count()\
.sort(desc('count'))\
.show(5, truncate=False)

+-------------------------------------------+-----+
|NM_PARTIDO                                 |count|
+-------------------------------------------+-----+
|PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO|231  |
|PARTIDO DOS TRABALHADORES                  |193  |
|PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA    |170  |
|PARTIDO SOCIAL DEMOCRÁTICO                 |120  |
|PARTIDO SOCIALISTA BRASILEIRO              |110  |
+-------------------------------------------+-----+
only showing top 5 rows



In [ ]:
spark.sql("select NM_PARTIDO, count(*) as count \
from consulta_cand_2014 where CD_SIT_TOT_TURNO in ('1','2','3') \
group by 1 order by 2 desc")\
.show(5, truncate=False)

+-------------------------------------------+-----+
|NM_PARTIDO                                 |count|
+-------------------------------------------+-----+
|PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO|231  |
|PARTIDO DOS TRABALHADORES                  |193  |
|PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA    |170  |
|PARTIDO SOCIAL DEMOCRÁTICO                 |120  |
|PARTIDO SOCIALISTA BRASILEIRO              |110  |
+-------------------------------------------+-----+
only showing top 5 rows



### **Qual o número de eleitos por partido em 2018 ordenando a saída pelo número de eleitos?**

Candidatos eleitos são aqueles com CD_SIT_TOT_TURNO igual a 1, 2 ou 3. O partido é definido pela coluna NM_PARTIDO.

In [ ]:
# Qual o número de eleitos por partido em 2018 ordenando a saída pelo número de eleitos?
consulta_cand_2018_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3')).groupBy(['NM_PARTIDO']).count().sort(desc('count'))\
.show(5, truncate=False)

+---------------------------------------+-----+
|NM_PARTIDO                             |count|
+---------------------------------------+-----+
|PARTIDO DOS TRABALHADORES              |153  |
|MOVIMENTO DEMOCRÁTICO BRASILEIRO       |149  |
|PARTIDO SOCIAL LIBERAL                 |142  |
|PROGRESSISTAS                          |122  |
|PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA|115  |
+---------------------------------------+-----+
only showing top 5 rows



In [ ]:
spark.sql("select NM_PARTIDO, count(*) as count from consulta_cand_2018 where CD_SIT_TOT_TURNO in ('1','2','3') group by 1 order by 2 desc")\
.show(5, truncate=False)

+---------------------------------------+-----+
|NM_PARTIDO                             |count|
+---------------------------------------+-----+
|PARTIDO DOS TRABALHADORES              |153  |
|MOVIMENTO DEMOCRÁTICO BRASILEIRO       |149  |
|PARTIDO SOCIAL LIBERAL                 |142  |
|PROGRESSISTAS                          |122  |
|PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA|115  |
+---------------------------------------+-----+
only showing top 5 rows



### **Quais candidatos eleitos em 2014 foram eleitos também em algum cargo eletivo em 2018?**

Candidatos eleitos são aqueles com CD_SIT_TOT_TURNO igual a 1, 2 ou 3.

**Dica**: O campo NR_CPF_CANDIDATO representa o cpf do candidato.

In [ ]:
# Quais candidatos eleitos em 2014 foram eleitos também em algum cargo eletivo em 2018? Por quê o atributo 'ST_REELEICAO' nem sempre reflete a realidade?
eleitos_2014_2018 = eleitos_2014.join(eleitos_2018, 'NR_CPF_CANDIDATO')
eleitos_2014_2018.show(5, truncate=False)

+----------------+-------------------------------+------------+-----------------+-------------------------------+------------+-----------------+------------+
|NR_CPF_CANDIDATO|NM_CANDIDATO                   |SQ_CANDIDATO|DS_CARGO         |NM_CANDIDATO                   |SQ_CANDIDATO|DS_CARGO         |ST_REELEICAO|
+----------------+-------------------------------+------------+-----------------+-------------------------------+------------+-----------------+------------+
|04245226676     |CÁSSIO ANTONIO FERREIRA SOARES |130000000603|DEPUTADO ESTADUAL|CASSIO ANTÔNIO FERREIRA SOARES |130000613113|DEPUTADO ESTADUAL|S           |
|13521039668     |MARIA MARGARIDA MARTINS SALOMÃO|130000000883|DEPUTADO FEDERAL |MARIA MARGARIDA MARTINS SALOMÃO|130000626964|DEPUTADO FEDERAL |S           |
|98042734700     |JOSIAS MARIO DA VITORIA        |80000000377 |DEPUTADO ESTADUAL|JOSIAS MARIO DA VITORIA        |80000611885 |DEPUTADO FEDERAL |N           |
|07567626420     |GETULIO NUNES DO REGO          |20

In [ ]:
eleitos_2014.createOrReplaceTempView("eleitos_2014")
eleitos_2018.createOrReplaceTempView("eleitos_2018")
spark.sql("select * from eleitos_2014 as e2014, eleitos_2018 as e2018 \
where e2014.NR_CPF_CANDIDATO = e2018.NR_CPF_CANDIDATO")\
.show(5, truncate=False)

+-------------------------------+----------------+------------+-----------------+-------------------------------+----------------+------------+-----------------+------------+
|NM_CANDIDATO                   |NR_CPF_CANDIDATO|SQ_CANDIDATO|DS_CARGO         |NM_CANDIDATO                   |NR_CPF_CANDIDATO|SQ_CANDIDATO|DS_CARGO         |ST_REELEICAO|
+-------------------------------+----------------+------------+-----------------+-------------------------------+----------------+------------+-----------------+------------+
|CÁSSIO ANTONIO FERREIRA SOARES |04245226676     |130000000603|DEPUTADO ESTADUAL|CASSIO ANTÔNIO FERREIRA SOARES |4245226676      |130000613113|DEPUTADO ESTADUAL|S           |
|MARIA MARGARIDA MARTINS SALOMÃO|13521039668     |130000000883|DEPUTADO FEDERAL |MARIA MARGARIDA MARTINS SALOMÃO|13521039668     |130000626964|DEPUTADO FEDERAL |S           |
|JOSIAS MARIO DA VITORIA        |98042734700     |80000000377 |DEPUTADO ESTADUAL|JOSIAS MARIO DA VITORIA        |98042734700 

## Quarta Etapa: Escrita do resultado

Nesta etapa iremos gerar uma saída com os eleitos em 2018 que estão no Top 50 com maior valor em reais de bens declarados

### **Nas eleições de 2018, quais eleitos estão no Top 50 com maior valor em Reais de bens declarados?**

Nesta etapa iremos identificar os candidatos eleitos em 2018 que estão no Top 50 de valor de bens declarados para analisar o impacto do poder financeiro na eleição. A saída será o nome do candidato, sigla do partido e o valor em reais de bens declarados na eleição de 2018. 


**Dicas**: 

- A coluna VR_BEM_CANDIDATO identifica o valor dos bens dos candidatos nas eleições de 2018.
- O VR_BEM_CANDIDATO está com vírgula ao invés de ponto no valor.
- A coluna SQ_CANDIDATO pode ser a coluna de junção entre os datasets bem_candidato_2018_df e consulta_cand_2018_df

**Observação**: para esta consulta **não** será necessário mapear em SQL.

In [ ]:
bem_candidato_2018_df = bem_candidato_2018_df.withColumn('VR_BEM_CANDIDATO', regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('double'))
t1 = bem_candidato_2018_df.sort(desc('VR_BEM_CANDIDATO')).limit(50)
t2 = consulta_cand_2018_df.filter(col('CD_SIT_TOT_TURNO').isin('1','2','3'))

In [ ]:
result = t1.join(t2, 'SQ_CANDIDATO').dropDuplicates(['NM_CANDIDATO'])\
.select("NM_CANDIDATO", "DS_CARGO", "SG_PARTIDO", "VR_BEM_CANDIDATO").sort(desc('VR_BEM_CANDIDATO'))

result.show(5, truncate=False)

### **Escrevendo o resultado no formato Parquet**

Abaixo faça a escrita do resultado da análise anterior dos campos nome do candidato, sigla do partido e o valor em reais de bens declarados na eleição de 2018. no formato **Parquet** em "/tmp/cand_bens.parquet"

In [ ]:
# Escrita no formato Parquet
result.write.parquet("/tmp/cand_bens.parquet", mode='overwrite')

In [ ]:
! ls /tmp/cand_bens.parquet

## Quinta Etapa: Exercício prático

Nesta etapa resolva o desafio proposto neste notebook é responder as perguntas abaixo para os datasets de eleições de 2022:

1. Quais candidatos foram eleitos em 2022 no Estado de Goiás?
2. Qual o número de eleitos por partido em 2022 no Estado de Goiás ordenando a saída pelo número de eleitos?
3. Quais candidatos eleitos em 2014 e 2018 foram eleitos também em algum cargo eletivo em 2022 no Estado de Goiás?
4. Nas eleições de 2022, quais eleitos estão no Top 50 com maior valor em Reais de bens declarados no Estado de Goiás? Escreva o resultado no formato Parquet.

In [ ]:
sh = """
# Baixa o arquivo que contém as informações dos candidatos de 2022
if [ ! -f eleicoes/consulta_candidato_2022_BRASIL.csv ]; then 
  wget https://github.com/savioteles/big_data/raw/master/etl/datasets/consulta_cand_2022_BRASIL.csv
  mv consulta_cand_2022_BRASIL.csv eleicoes
fi

# Baixa o arquivo que contém as informações dos bens dos candidatos de 2022
if [ ! -f eleicoes/bem_candidato_2022_BRASIL.csv ]; then 
  wget https://github.com/savioteles/big_data/raw/master/etl/datasets/bem_candidato_2022_BRASIL.csv
  mv bem_candidato_2022_BRASIL.csv eleicoes
fi
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

--2022-10-25 23:51:20--  https://github.com/savioteles/big_data/raw/master/etl/datasets/consulta_cand_2022_BRASIL.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/savioteles/big_data/master/etl/datasets/consulta_cand_2022_BRASIL.csv [following]
--2022-10-25 23:51:20--  https://raw.githubusercontent.com/savioteles/big_data/master/etl/datasets/consulta_cand_2022_BRASIL.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21758282 (21M) [text/plain]
Saving to: ‘consulta_cand_2022_BRASIL.csv’

consulta_cand_2022_ 100%[===================>]  20.75M  --.-KB/s    in 0.06s   

2022-10-25 23:51:20 (376

In [ ]:
# Leitura dos arquivos com informações dos bens dos candidatos em 2022
bem_candidato_2022_df = spark.read.csv('file:///content/eleicoes/bem_candidato_2022_BRASIL.csv', header=True, encoding='Latin1', sep=';', schema=bem_candidato_schema, dateFormat='dd/MM/yyyy')

In [ ]:
# Leitura dos arquivos da candidatura dos anos de 2022
consulta_cand_2022_df = spark.read.csv('file:///content/eleicoes/consulta_cand_2022_BRASIL.csv', header=True, encoding='Latin1', sep=';', inferSchema=True, timestampFormat='dd/MM/yyyy')

In [ ]:
consulta_cand_2022_df = consulta_cand_2022_df.withColumn('DS_SIT_TOT_TURNO', regexp_replace('DS_SIT_TOT_TURNO', '#NULO#', ''))

In [ ]:
# candidatos 2022
consulta_cand_2022_df = consulta_cand_2022_df.withColumn('DS_SIT_TOT_TURNO', regexp_replace('DS_SIT_TOT_TURNO', '#NULO#', ''))
consulta_cand_2022_df = <faça o mesmo tratamento acima para a coluna 'NM_SOCIAL_CANDIDATO'>

In [ ]:
# bens dos candidatos de 2022
bem_candidato_2022_df = <faça o mesmo tratamento acima na coluna 'DS_BEM_CANDIDATO'>

### **Quais candidatos foram eleitos em 2022 no Estado de Goiás?**

In [ ]:
# #Quais candidatos foram eleitos em 2022 no Estado de Goiás?
eleitos_2022 = consulta_cand_2022_df\
                .filter("SG_UF = 'GO' and CD_SIT_TOT_TURNO in (1,2,3)")\
                .select('NM_CANDIDATO', 'SG_UF', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'DS_CARGO', 'ST_REELEICAO')\

eleitos_2022.show(5, truncate=False)

+------------------------------+-----+----------------+------------+-----------------+------------+
|NM_CANDIDATO                  |SG_UF|NR_CPF_CANDIDATO|SQ_CANDIDATO|DS_CARGO         |ST_REELEICAO|
+------------------------------+-----+----------------+------------+-----------------+------------+
|RENATO MENEZES DE CASTRO      |GO   |78475317120     |90001648408 |DEPUTADO ESTADUAL|N           |
|MAGDA MOFATTO HON             |GO   |13205285115     |90001652511 |DEPUTADO FEDERAL |S           |
|RICARDO QUIRINO DOS SANTOS    |GO   |83217037715     |90001653000 |DEPUTADO ESTADUAL|N           |
|JAMIL SEBBA CALIFE            |GO   |92219322149     |90001712193 |DEPUTADO ESTADUAL|N           |
|AMILTON BATISTA DE FARIA FILHO|GO   |218004133       |90001653191 |DEPUTADO ESTADUAL|S           |
+------------------------------+-----+----------------+------------+-----------------+------------+
only showing top 5 rows



### **Qual o número de eleitos por partido em 2022 no Estado de Goiás ordenando a saída pelo número de eleitos?**

In [ ]:
# Qual o número de eleitos por partido em 2022 ordenando a saída pelo número de eleitos?
num_cand_eleitos_partido = <codigo aqui>
num_cand_eleitos_partido.show(5, truncate=False)

### **Quais candidatos eleitos em 2014 e 2018 foram eleitos também em algum cargo eletivo em 2022 no Estado de Goiás?**

In [ ]:
# Quais candidatos eleitos em 2014 foram eleitos também em algum cargo eletivo em 2018?
eleitos_2014_2018_2022 = <faça join entre eleitos_2022 e eleitos_2014_2018>
eleitos_2014_2018_2022.show(5, truncate=False)

### **Nas eleições de 2022, quais eleitos estão no Top 50 com maior valor em Reais de bens declarados no Estado de Goiás? Escreva o resultado no formato Parquet.**

In [ ]:
bem_candidato_2022_df = <faça a conversão da coluna 'VR_BEM_CANDIDATO' para double lembrando de substituir ',' por '.'>
t1 = <filtre no bem_candidato_2022_df  pelo estado de Goiás e ordene pela coluna 'VR_BEM_CANDIDATO' na ordem decrescente pegando os 50 primeiros>
t2 = <filtre no consulta_cand_2022_df pela coluna CD_SIT_TOT_TURNO igual a 1,2 ou 3 e pelo estado de Goiás>

In [ ]:
result = <faça o join entre t1 e t2 e remova os duplicados ordenando pela coluna VR_BEM_CANDIDATO>

result.show(5, truncate=False)

In [ ]:
<escreva o resultado em "file:///tmp/bens_cand_2022.parquet">